In [1]:
from time import time
import os

import pandas as pd
from gensim.models.keyedvectors import KeyedVectors

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

from preprocess import prep_sentence_couples
# from model import SiameseLSTM


# Variables ###################################################################
checkpoint_path = '/home/erik/Desktop/sia_checkpoints'
lr = 20

Using TensorFlow backend.


In [2]:
torch.FloatTensor([0])[0]

0.0

In [2]:
# Pre-process data ############################################################
print('Pre-processing data')
word_vectors = KeyedVectors.load_word2vec_format(
    '~/datasets/GoogleNews-vectors-negative300.bin',
    binary=True)

train = pd.read_csv('../small_data/train.csv')
test = pd.read_csv('../small_data/test.csv')

X_train1, X_train2 = prep_sentence_couples(train['str1'],
                                           train['str2'],
                                           word_vectors)
X_test1, X_test2 = prep_sentence_couples(test['str1'],
                                         test['str2'],
                                         word_vectors)

y_train = train['paraphrase'].values
y_test = test['paraphrase'].values

# pytorch-ify the vectors######################################################
X_train1 = torch.from_numpy(X_train1).float()
X_train2 = torch.from_numpy(X_train2).float()

X_test1 = torch.from_numpy(X_test1).float()
X_test2 = torch.from_numpy(X_test2).float()

y_train = torch.from_numpy(y_train).float()
y_test = torch.from_numpy(y_test).float()

Pre-processing data


In [ ]:
model = SiameseLSTM(300)
optimizer = optim.Adam(model.parameters())
criterion = nn.L1Loss()

In [6]:
def get_batch(x1, x2, y, i, batch_size, evaluate=False):
    batch_size = min(batch_size, len(y) - 1 - i)
    x1i = Variable(x1[i:i+batch_size], volatile=evaluate)
    x2i = Variable(x2[i:i+batch_size], volatile=evaluate)
    yi = Variable(y[i:i+batch_size])
    return (x1i, x2i), yi

In [ ]:
# Train the model #############################################################
def train_epoch(epoch, batch_size, x1, x2, y):
    start_time = time()
    total_loss = 0

    model.train()

    n_items = len(y)

    for n, i in enumerate(range(0, y.size(0)-1, batch_size)):
        model.zero_grad()
        h0, h1 = model.init_hidden()
#         print(h0, h1)
        (x1i, x2i), yi = get_batch(x1, x2, y, i, batch_size)
        print(x1i, x2i)
        # calculate error and backprop
        output, h0, h1 = model(x1i, x2i, h0, h1)
        
        loss = criterion(output, yi)
        
        loss.backward()
        optimizer.step()

        # helps combat vanishing gradients
        nn.utils.clip_grad_norm(model.parameters(), 0.25)
        for p in model.parameters():
            p.data.add_(-lr, p.grad.data)

        # track metrics
        total_loss += loss.data

    avg_loss = float(total_loss) / n_items

    time_taken = time() - start_time
    print('{}| time taken: {} total loss: {} avg loss: {}'.format(time_taken,
                                                                  total_loss,
                                                                  avg_loss))


def evaluate(x1, x2, y):
    model.eval()
    total_loss = 0
    for i, (x1i, x2i, yi) in enumerate(zip(x1, x2, y)):
        output, h0, h1 = model(x1i, x2i)
        total_loss += len(y) * criterion(output, yi).data

    return total_loss / float(len(y))


def checkpoint(epoch, model):
    with open(
        os.path.join(checkpoint_path, 'siamese_lstm_epoch{}.pt'.format(epoch)),
        'wb'
    ) as f:
        torch.save(model, f)


In [ ]:
try:
    for epoch in range(1, 1000):
        train_epoch(epoch, 32, X_train1, X_train2, y_train)

        eval_loss = evaluate(X_test1, X_test2, y_test)
        print('epoch {} eval loss: {:.6f}'.format(epoch, eval_loss))

        if not best_val_loss or best_val_loss > eval_loss:
            best_val_loss = eval_loss
        else:
            lr /= 4.0

        if epoch % 5 == 0:
            checkpoint(epoch, model)


except KeyboardInterrupt:
    print('Keyboard interupt received, saving model and finishing')
    checkpoint('exit', model)

In [ ]:
def cosine_similarity(x1, x2, dim=0, eps=1e-8):
    w12 = torch.sum(x1 * x2, dim)
    w1 = torch.norm(x1, 2, dim)
    w2 = torch.norm(x2, 2, dim)
    return (w12 / (w1 * w2).clamp(min=eps)).squeeze()


class SiameseLSTM(nn.Module):
    def __init__(self, input_size=300, hidden_size=100, num_layers=2, batch_size=32, dropout=0.3):
        super(SiameseLSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.batch_size = batch_size

        # will need this later
        # self.dropout = nn.Dropout(0.3)

        # LSTM stuff
        self.rnn0 = nn.LSTM(input_size, hidden_size, num_layers)
        self.rnn1 = nn.LSTM(input_size, hidden_size, num_layers)

        # Linear layers so shapes play nice
        # self.l0 = nn.Linear(200, 100)
        # self.l1 = nn.Linear(200, 100)

    def forward(self, x0, x1, h0, h1):
        x0, h0 = self.rnn0(x0, h0)
        x1, h1 = self.rnn1(x1, h1)

        # x0 = self.l0(x0.view(-1, 200))
        # x1 = self.l1(x1.view(-1, 200))

#         d = cosine_similarity(x0.view(1, 32, -1), x1.view(1, 32, -1))
        return x0, h0, h1

    
    def init_hidden(self):
        return Variable(torch.zeros((self.num_layers, self.batch_size, self.hidden_size))), Variable(torch.zeros((self.num_layers, self.batch_size, self.hidden_size)))

In [49]:
import numpy as np

def cosine_similarity(x1, x2, dim=1, eps=1e-8):
    print(x1.size(), x2.size())
    w12 = torch.sum(x1 * x2, dim)
    w1 = torch.norm(x1, 2, dim)
    w2 = torch.norm(x2, 2, dim)
    d = (w12 / (w1 * w2).clamp(min=eps)).squeeze()
    print('distance', d, d.size())
    return d

class RNN(nn.Module):
    def __init__(self, input_size=300, hidden_size=200, num_layers=3,
                 batch_size=10):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = batch_size
        self.batch_size = batch_size

        self.rnn1 = nn.RNN(input_size=self.input_size, hidden_size=self.hidden_size,
                          num_layers=self.num_layers, batch_first=True)

        self.rnn2 = nn.RNN(input_size=self.input_size, hidden_size=self.hidden_size,
                  num_layers=self.num_layers, batch_first=True)
        # The last layer is applied on the last output only of the RNN (not like
        # TimeDistributedDense in Keras)
        self.linear_layer1 = nn.Linear(self.hidden_size, self.output_size)
        self.linear_layer2 = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, x1, x2, h1, h2):
        out1, h1 = self.rnn1(x1, h2)
        out2, h2 = self.rnn1(x1, h2)
        print('out1 shape', out1.size())
        
        in_linear1 = out1[:, -1, :]
        in_linear2 = out2[:, -1, :]
        print('lin1 shape', in_linear1.size())
#         final_output1 = self.linear_layer1(in_linear1)
#         final_output2 = self.linear_layer2(in_linear2)
        
#         print('fwd:,', final_output1.size(), final_output2.size())
        
        dist = cosine_similarity(in_linear1, in_linear1)
        return dist, h1, h2

    def init_hidden(self):
        return Variable(torch.zeros((self.num_layers, self.batch_size, self.hidden_size))).cuda(), Variable(torch.zeros((self.num_layers, self.batch_size, self.hidden_size))).cuda()


In [32]:
def cosine_similarity(x1, x2, dim=0, eps=1e-8):
    w12 = torch.sum(x1 * x2, dim)
    w1 = torch.norm(x1, 2, dim)
    w2 = torch.norm(x2, 2, dim)
    d = (w12 / (w1 * w2).clamp(min=eps)).squeeze()
    print('distance', d)
    return d

# cosine_similarity(torch.Tensor([[0,2,3],[1,2,3]]), torch.Tensor([[1,2,3],[1,2,3]]))
torch.Tensor([[1],[1],[3],[4]]).size()

cosine_similarity(torch.Tensor([[1],[1],[3],[4]]), torch.Tensor([[1],[1],[3],[1]]))

distance 
 0.8333
[torch.FloatTensor of size 1]




 0.8333
[torch.FloatTensor of size 1]

In [50]:
def get_batch(X1, X2, y, i, evaluation=False):
    global batch_size
    batch_size = min(batch_size, len(y) - 1 - i)
    xi1 = Variable(X1[i:i+batch_size], volatile=evaluation)
    xi2 = Variable(X2[i:i+batch_size], volatile=evaluation)
    yi = Variable(y[i:i+batch_size])
    return xi1, xi2, yi


def train(X1, X2, y):
    rnn.train()
    total_loss = 0
    for batch, i in enumerate(range(0, y.size(0) - 1, batch_size)):
        h1, h2 = rnn.init_hidden()
        if (len(X1) - i) < batch_size:
            continue
        x1i, x2i, targets = get_batch(X1, X2, y, i)

        output, h1, h2 = rnn(x1i, x2i, h1, h2)

        optimizer.zero_grad()
        loss = loss_fn(output, targets)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.data[0]

    return total_loss

def calc_accuracy(y, y_hat):
#     print(type(y))
    y, y_hat = y.data.clone().cpu().numpy(), y_hat.data.clone().cpu().numpy()
    
    dif = y - y_hat
    wrong = 0.0
    for d in dif:
        if d == 0:
            wrong += 1

    acc = float(wrong) / len(y_hat)
    std = np.std(y_hat)
    
    return 1 - acc, std
        

def evaluate(X1, X2, y):
    rnn.eval()
    total_loss = 0.0
    batches_ran = 0.0
    total_acc = 0.0
    for batch, i in enumerate(range(0, y.size(0) - 1, batch_size)):
        h1, h2 = rnn.init_hidden()
        if (len(X1) - i) < batch_size:
            continue
        x1i, x2i, targets = get_batch(X1, X2, y, i, evaluation=True)

        output, h1, h2 = rnn(x1i, x2i, h1, h2)

        total_loss += len(y) * loss_fn(output, targets).data
        acc, std = calc_accuracy(targets, output)
        total_acc += acc
        batches_ran += 1

    avg_loss = total_loss / batches_ran
    return {'avg_loss': avg_loss[0], 'avg_acc': total_acc / batches_ran, 'std_of_answer': std}




In [51]:
batch_size = 32

USE_CUDA = True

rnn = RNN(input_size=300, hidden_size=200, num_layers=3, batch_size=32).cuda()

if USE_CUDA:
    X_train1 = X_train1.cuda()
    X_train2 = X_train2.cuda()
    y_train = y_train.cuda()
    X_test1 = X_test1.cuda()
    X_test2 = X_test2.cuda()
    y_test = y_test.cuda()
    
    rnn.cuda()

loss_fn = nn.MSELoss()
optimizer = optim.RMSprop(rnn.parameters())
print('starting')

for epoch in range(20):
    train_loss = train(X_train1, X_train2, y_train)
    print("Epoch {} -- train loss = {}".format(epoch, train_loss))
    
    eval_results = evaluate(X_test1, X_test2, y_test)
    print('-- Eval stats:\n \t', eval_results)
    print("-"*100)

starting
out1 shape torch.Size([32, 35, 200])
lin1 shape torch.Size([32, 200])
torch.Size([32, 200]) torch.Size([32, 200])
distance Variable containing:
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
[torch.cuda.FloatTensor of size 32 (GPU 0)]
 torch.Size([32])
out1 shape torch.Size([32, 35, 200])
lin1 shape torch.Size([32, 200])
torch.Size([32, 200]) torch.Size([32, 200])
distance Variable containing:
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
[torch.cuda.FloatTensor of size 32 (GPU 0)]
 torch.Size([32])
out1 shape torch.Size([32, 35, 200])
lin1 shape torch.Size([32, 200

out1 shape torch.Size([32, 35, 200])
lin1 shape torch.Size([32, 200])
torch.Size([32, 200]) torch.Size([32, 200])
distance Variable containing:
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
[torch.cuda.FloatTensor of size 32 (GPU 0)]
 torch.Size([32])
out1 shape torch.Size([32, 35, 200])
lin1 shape torch.Size([32, 200])
torch.Size([32, 200]) torch.Size([32, 200])
distance Variable containing:
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
[torch.cuda.FloatTensor of size 32 (GPU 0)]
 torch.Size([32])
out1 shape torch.Size([32, 35, 200])
lin1 shape torch.Size([32, 200])
torch.

out1 shape torch.Size([32, 35, 200])
lin1 shape torch.Size([32, 200])
torch.Size([32, 200]) torch.Size([32, 200])
distance Variable containing:
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
[torch.cuda.FloatTensor of size 32 (GPU 0)]
 torch.Size([32])
out1 shape torch.Size([32, 35, 200])
lin1 shape torch.Size([32, 200])
torch.Size([32, 200]) torch.Size([32, 200])
distance Variable containing:
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
[torch.cuda.FloatTensor of size 32 (GPU 0)]
 torch.Size([32])
out1 shape torch.Size([32, 35, 200])
lin1 shape torch.Size([32, 200])
torch.

out1 shape torch.Size([32, 35, 200])
lin1 shape torch.Size([32, 200])
torch.Size([32, 200]) torch.Size([32, 200])
distance Variable containing:
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
[torch.cuda.FloatTensor of size 32 (GPU 0)]
 torch.Size([32])
out1 shape torch.Size([32, 35, 200])
lin1 shape torch.Size([32, 200])
torch.Size([32, 200]) torch.Size([32, 200])
distance Variable containing:
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
[torch.cuda.FloatTensor of size 32 (GPU 0)]
 torch.Size([32])
out1 shape torch.Size([32, 35, 200])
lin1 shape torch.Size([32, 200])
torch.

out1 shape torch.Size([32, 35, 200])
lin1 shape torch.Size([32, 200])
torch.Size([32, 200]) torch.Size([32, 200])
distance Variable containing:
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
[torch.cuda.FloatTensor of size 32 (GPU 0)]
 torch.Size([32])
out1 shape torch.Size([32, 35, 200])
lin1 shape torch.Size([32, 200])
torch.Size([32, 200]) torch.Size([32, 200])
distance Variable containing:
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
[torch.cuda.FloatTensor of size 32 (GPU 0)]
 torch.Size([32])
out1 shape torch.Size([32, 35, 200])
lin1 shape torch.Size([32, 200])
torch.

...

lin1 shape torch.Size([32, 200])
torch.Size([32, 200]) torch.Size([32, 200])
distance Variable containing:
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
[torch.cuda.FloatTensor of size 32 (GPU 0)]
 torch.Size([32])
out1 shape torch.Size([32, 35, 200])
lin1 shape torch.Size([32, 200])
torch.Size([32, 200]) torch.Size([32, 200])
distance Variable containing:
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
[torch.cuda.FloatTensor of size 32 (GPU 0)]
 torch.Size([32])
out1 shape torch.Size([32, 35, 200])
lin1 shape torch.Size([32, 200])
torch.Size([32, 200]) torch.Size([32, 200])

distance Variable containing:
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
[torch.cuda.FloatTensor of size 32 (GPU 0)]
 torch.Size([32])
out1 shape torch.Size([32, 35, 200])
lin1 shape torch.Size([32, 200])
torch.Size([32, 200]) torch.Size([32, 200])
distance Variable containing:
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
[torch.cuda.FloatTensor of size 32 (GPU 0)]
 torch.Size([32])
out1 shape torch.Size([32, 35, 200])
lin1 shape torch.Size([32, 200])
torch.Size([32, 200]) torch.Size([32, 200])
distance Variable containing:
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.000

out1 shape torch.Size([32, 35, 200])
lin1 shape torch.Size([32, 200])
torch.Size([32, 200]) torch.Size([32, 200])
distance Variable containing:
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
[torch.cuda.FloatTensor of size 32 (GPU 0)]
 torch.Size([32])
out1 shape torch.Size([32, 35, 200])
lin1 shape torch.Size([32, 200])
torch.Size([32, 200]) torch.Size([32, 200])
distance Variable containing:
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
[torch.cuda.FloatTensor of size 32 (GPU 0)]
 torch.Size([32])
out1 shape torch.Size([32, 35, 200])
lin1 shape torch.Size([32, 200])
torch.

out1 shape torch.Size([32, 35, 200])
lin1 shape torch.Size([32, 200])
torch.Size([32, 200]) torch.Size([32, 200])
distance Variable containing:
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
[torch.cuda.FloatTensor of size 32 (GPU 0)]
 torch.Size([32])
out1 shape torch.Size([32, 35, 200])
lin1 shape torch.Size([32, 200])
torch.Size([32, 200]) torch.Size([32, 200])
distance Variable containing:
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
[torch.cuda.FloatTensor of size 32 (GPU 0)]
 torch.Size([32])


KeyboardInterrupt: 